In [1]:
import pandas as pd
import numpy as np

# Run initial dropping of columns

In [2]:
def clean_initial(path, output_path):
    name, _ = path.split(".")
    df = pd.read_csv(path)

    # delete columns with alpha data
    del df['respondent_id'] # save elsewhere 
    del df['agency_name']
    del df['agency_abbr']
    del df['loan_type_name']
    del df['property_type_name']
    del df['loan_purpose_name']
    del df['owner_occupancy_name']
    del df['preapproval_name']
    del df['action_taken_name']
    del df['msamd_name']
    del df['state_name']
    del df['county_name']
    del df['applicant_ethnicity_name']
    del df['co_applicant_ethnicity_name']
    del df['applicant_race_name_1']
    del df['applicant_race_name_2']
    del df['applicant_race_name_3']
    del df['applicant_race_name_4']
    del df['applicant_race_name_5']
    del df['co_applicant_race_name_1']
    del df['co_applicant_race_name_2']
    del df['co_applicant_race_name_3']
    del df['co_applicant_race_name_4']
    del df['co_applicant_race_name_5']
    del df['applicant_sex_name']
    del df['co_applicant_sex_name']
    del df['purchaser_type_name']
    del df['denial_reason_name_1']
    del df['denial_reason_name_2']
    del df['denial_reason_name_3']
    del df['hoepa_status_name']
    del df['lien_status_name']
    del df['edit_status_name']
    del df['application_date_indicator'] # a 2004 date indicator
    
    # delete proxies
    del df['lien_status'] # also a proxy
    del df['hoepa_status'] # only for originated loans, so proxy
    
    # some missing state info; delete those since we need that to subdivide
    # some missing state info; remove those rows
    df = df.dropna(subset=['state_abbr', 'state_code'])
    return df

In [3]:
data_fn = "hmda_2007_nationwide_all-records_labels.csv"

In [4]:
clean_df = clean_initial(data_fn, "./")
m, n = clean_df.shape
m, n

<ipython-input-2-feb12b8277c2>:3: DtypeWarning: Columns (34,36,38,44,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


(25916848, 42)

In [5]:
clean_df.head()

,as_of_year,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,msamd,...,denial_reason_3,rate_spread,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,2007,7,1,2,1,1,49,2,1,NaN,...,NaN,NaN,NaN,13352,3427.0,20.129999,45700.0,117.680000,1070.0,1326.0
1,2007,4,1,1,1,1,500,3,6,NaN,...,NaN,NaN,NaN,2897,3574.0,19.219999,52400.0,123.230003,884.0,1266.0
2,2007,7,1,2,1,1,66,2,1,NaN,...,NaN,NaN,NaN,13357,5628.0,24.270000,45700.0,116.000000,1397.0,1828.0
3,2007,7,1,1,3,2,60,3,3,NaN,...,NaN,NaN,NaN,13374,6668.0,14.230000,45700.0,102.919998,1972.0,2787.0
4,2007,7,1,1,1,1,143,2,1,NaN,...,NaN,NaN,NaN,13360,5628.0,24.270000,45700.0,116.000000,1397.0,1828.0


In [6]:
# some missing loan amount; need this
clean_df = clean_df.dropna(subset=['loan_amount_000s'])
m, n = clean_df.shape
m, n

(25916848, 42)

# First NaN Filter

In [7]:
clean_df.columns[clean_df.isna().any()].tolist()

['msamd',
 'county_code',
 'census_tract_number',
 'applicant_race_2',
 'applicant_race_3',
 'applicant_race_4',
 'applicant_race_5',
 'co_applicant_race_2',
 'co_applicant_race_3',
 'co_applicant_race_4',
 'co_applicant_race_5',
 'applicant_income_000s',
 'denial_reason_1',
 'denial_reason_2',
 'denial_reason_3',
 'rate_spread',
 'edit_status',
 'population',
 'minority_population',
 'hud_median_family_income',
 'tract_to_msamd_income',
 'number_of_owner_occupied_units',
 'number_of_1_to_4_family_units']

In [8]:
# some missing crucial census info; remove
clean_df = clean_df.dropna(subset=['county_code'])
clean_df = clean_df.dropna(subset=['census_tract_number'])
clean_df = clean_df.dropna(subset=['population'])
clean_df = clean_df.dropna(subset=['minority_population'])
clean_df = clean_df.dropna(subset=['hud_median_family_income'])
clean_df = clean_df.dropna(subset=['tract_to_msamd_income'])
clean_df = clean_df.dropna(subset=['number_of_owner_occupied_units'])
clean_df = clean_df.dropna(subset=['number_of_1_to_4_family_units'])

m, n = clean_df.shape
m, n

(25748757, 42)

# Second NaN Filter

In [9]:
clean_df.columns[clean_df.isna().any()].tolist()

['msamd',
 'applicant_race_2',
 'applicant_race_3',
 'applicant_race_4',
 'applicant_race_5',
 'co_applicant_race_2',
 'co_applicant_race_3',
 'co_applicant_race_4',
 'co_applicant_race_5',
 'applicant_income_000s',
 'denial_reason_1',
 'denial_reason_2',
 'denial_reason_3',
 'rate_spread',
 'edit_status']

In [10]:
print(clean_df.loc[clean_df['msamd'].isna()].index)
# keep msamd; about 7% missing it, can filter out those examples or remove column, leave up to user

print(clean_df.loc[clean_df['applicant_income_000s'].isna()].index)
# keep applicant_income, but note that about 12% missing this; can also leave up to user

Int64Index([       0,        1,        2,        3,        4,        5,
                   6,        7,        8,        9,
            ...
            26605618, 26605631, 26605648, 26605651, 26605660, 26605670,
            26605674, 26605676, 26605677, 26605693],
           dtype='int64', length=3082358)
Int64Index([       1,       27,      110,      134,      140,      178,
                 280,      344,      348,      351,
            ...
            26605554, 26605565, 26605580, 26605616, 26605619, 26605639,
            26605656, 26605663, 26605678, 26605687],
           dtype='int64', length=2485820)


In [11]:
# drop rows where msamd is NA
msamd_nan = clean_df.loc[clean_df['msamd'].isna()].index.tolist()
clean_df = clean_df.drop(msamd_nan)

In [12]:
# drop rows where applicant_income_000s is NaN  
income_nan = clean_df.loc[clean_df['applicant_income_000s'].isna()].index.tolist()
clean_df = clean_df.drop(income_nan)

# Third NaN Filter

In [13]:
print(len(clean_df.loc[~clean_df['rate_spread'].isna()].index))

1518303


In [14]:
# almost everything is missing rate spread; delete it
del clean_df['rate_spread']

In [15]:
print(len(clean_df.loc[clean_df['sequence_number'].isna()].index))

0


In [16]:
print(len(clean_df.loc[~clean_df['edit_status'].isna()].index))

3867218


In [17]:
# significant number of loans have edit status set; valid statuses are 6, 7, 8; set to 0 if NA
clean_df["edit_status"] = clean_df["edit_status"].fillna(0)
print(len(clean_df.loc[clean_df['edit_status'].isna()].index))

0


# Fourth NaN Check

In [18]:
clean_df.columns[clean_df.isna().any()].tolist()

['applicant_race_2',
 'applicant_race_3',
 'applicant_race_4',
 'applicant_race_5',
 'co_applicant_race_2',
 'co_applicant_race_3',
 'co_applicant_race_4',
 'co_applicant_race_5',
 'denial_reason_1',
 'denial_reason_2',
 'denial_reason_3']

In [19]:
clean_df.columns[~clean_df.isna().any()].tolist()

['as_of_year',
 'agency_code',
 'loan_type',
 'property_type',
 'loan_purpose',
 'owner_occupancy',
 'loan_amount_000s',
 'preapproval',
 'action_taken',
 'msamd',
 'state_abbr',
 'state_code',
 'county_code',
 'census_tract_number',
 'applicant_ethnicity',
 'co_applicant_ethnicity',
 'applicant_race_1',
 'co_applicant_race_1',
 'applicant_sex',
 'co_applicant_sex',
 'applicant_income_000s',
 'purchaser_type',
 'edit_status',
 'sequence_number',
 'population',
 'minority_population',
 'hud_median_family_income',
 'tract_to_msamd_income',
 'number_of_owner_occupied_units',
 'number_of_1_to_4_family_units']

# Co-applicant binarization

In [20]:
clean_df['has_co_applicant'] = np.nan
# these are the race ids that correspond to an identified race
co_app_ids = clean_df.index[clean_df['co_applicant_race_1'].isin([1,2,3,4,5])].tolist()
# this is the id for "no co-applicant"
no_co_app_ids = clean_df.index[clean_df['co_applicant_race_1'] == 8].tolist()
clean_df.loc[co_app_ids,['has_co_applicant']] = [1]
clean_df.loc[no_co_app_ids,['has_co_applicant']] = [0]

In [21]:
clean_df.head()

,as_of_year,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,msamd,...,denial_reason_3,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,has_co_applicant
23,2007,4,2,1,3,1,70,3,1,41700.0,...,NaN,6.0,169961,7078.0,78.739998,53700.0,63.639999,1118.0,1958.0,1.0
31,2007,1,1,1,3,1,24,3,6,17460.0,...,NaN,0.0,226435,4355.0,10.820000,60700.0,195.559998,1375.0,1443.0,0.0
56,2007,4,1,1,1,1,215,3,1,40060.0,...,NaN,6.0,1025503,9484.0,30.070000,68700.0,103.760002,3040.0,3658.0,1.0
72,2007,4,1,1,3,1,213,3,6,40900.0,...,NaN,0.0,1083940,4051.0,62.770000,67200.0,96.980003,936.0,1151.0,0.0
86,2007,3,1,1,3,1,323,3,1,14860.0,...,NaN,0.0,407,6874.0,36.029999,93400.0,75.889999,1451.0,1789.0,1.0


In [22]:
clean_df.columns[clean_df.isna().any()].tolist() # should have 'has_co_applicant'; demographic; denial

['applicant_race_2',
 'applicant_race_3',
 'applicant_race_4',
 'applicant_race_5',
 'co_applicant_race_2',
 'co_applicant_race_3',
 'co_applicant_race_4',
 'co_applicant_race_5',
 'denial_reason_1',
 'denial_reason_2',
 'denial_reason_3',
 'has_co_applicant']

# Save into one giant file

In [23]:
output_name = "hmda_2007_bulk.csv"
clean_df.to_csv(output_name, index=False)

# Split into different dataframes and save

## Target

In [ ]:
target_df = clean_df[['action_taken',
                      'denial_reason_1', 
                      'denial_reason_2', 
                      'denial_reason_3']]

In [ ]:
target_df.head()

In [ ]:
m, n = target_df.shape
m, n

In [ ]:
base_out = "hmda_{yr}_all_states_".format(yr=2007)

target_path = "{b}target_final.csv".format(b=base_out)
target_df.to_csv(target_path)

In [ ]:
del clean_df['denial_reason_1']
del clean_df['denial_reason_2']
del clean_df['denial_reason_3']
del clean_df['action_taken']

## Protected attributes

In [ ]:
clean_df.columns[clean_df.isna().any()].tolist() 

In [ ]:
protected_attr_cols = ['applicant_ethnicity',
                       'applicant_race_1',
                       'applicant_race_2',
                       'applicant_race_3',
                       'applicant_race_4',
                       'applicant_race_5',
                       'applicant_sex',
                       'co_applicant_ethnicity',
                       'co_applicant_race_1', 
                       'co_applicant_race_2', 
                       'co_applicant_race_3', 
                       'co_applicant_race_4', 
                       'co_applicant_race_5',
                       'co_applicant_sex']

g_df = clean_df[protected_attr_cols]

In [ ]:
g_df.head()

In [ ]:
m, n = g_df.shape
m, n

In [ ]:
g_path = "{b}protected_final.csv".format(b=base_out)
g_df.to_csv(g_path)

In [ ]:
for c in protected_attr_cols:
    del clean_df[c]

In [ ]:
m, n = clean_df.shape
m, n

## Features

In [ ]:
clean_df.columns[clean_df.isna().any()].tolist() # should be empty

In [ ]:
features_path = "{b}features_final.csv".format(b=base_out)
clean_df.to_csv(features_path)

In [ ]:
m, n = df.shape
m, n

In [ ]:
m, n = g_df.shape
m, n

In [ ]:
m, n = target_df.shape
m, n
target_df